# DataSimp How to Notebook
Welcome. This notebook is set up to walk a new user through how to use DataSimp.py to simplify the data you are using. The process is done assuming you are looking for binary stars using radial velocity data.

#### 1. Load in DataSimp.py
DataSimp is a class object that has all of the programs to simplify the data tables. The three variabled need to be the path to the allStar file, the path to the allVisit file, and the path to the iscochrone table file

In [10]:
%run DataSimp.py
dsimp = DataSimp('/home/cody/Binary Search/allStar-l31c.2.fits',
                 '/home/cody/Binary Search/allVisit-l31c.2.fits',
                 '/home/cody/Binary Search/parsec_decamsdss_all.fits.gz')

#### 2. Get the data from the fits files

In [4]:
all_average, all_visit, iso = dsimp.get_data()
len(all_visit)

1054381

You now have 3 objects of type "astropy.io.fits.fitsrec.FITS_rec". It's advised that you leave them like this for as long as you can. Turnning these into astropy Tables will eat up a lot of your memory and has frequently caused my computer to crash.

#### 3. Cut down all_visit
all_visit will often have radial velocity measurments with unphysical values. We first eliminate these, with the option of also eliminating observations with low SNR levels. Inputs to controle SNR and their default values,
    
    snr_cut = Ture
    snr_limit = 5

In [5]:
all_visit = dsimp.cuts(all_visit)

#### 4. Cut down on number of visits
Removed rows from all_average based on the "NVISITS" comlumn value. Only want to consider stars that were seen at least a sertain number of times. 

Also cutting down the size of all_average to 1000 stars just to save time. This is just an example case so there's no reason to spend time doing this for everything

In [6]:
print(len(all_average))
all_average = dsimp.nvisits_cut(all_average[0:5000], 5)
print(len(all_average))

277371
1265


#### 5. Red Gaint's
Makes a cut to all_average using the criteria that the star falls into the red giant group. The requirment comes from Troup et. al. (2016)

$log(g) \le 3.7 + 0.1$[FE/H]

$T_{eff} \le 5500$K

In [7]:
all_average = dsimp.rg_cut(all_average)
len(all_average)

600

#### 6. Isochrone fits. 
When finding the binary fraction we only want to keep stars that we can have good estimates for Mass, and Radius, as these will be the most impactfull when determaining if it's a binary or not. 

It looks for a similar fit using [Fe/H], LOGG, and TEFF. If no good fit is found then it removes the row from all_average. The program adds columns to all_average for the mean, median, and std of the good fits.

Print statments are added to see how far along the process is, but they can eassaly be removed.

In [8]:
len(all_average)

600

In [11]:
all_average = dsimp.iso_fit(all_average, iso)

Done with 0.000000
No good fits in 3 sigma of erros. Star 2M00002388+6151472, row 7
No good fits in 3 sigma of erros. Star 2M00012599+7010526, row 32
No good fits in 3 sigma of erros. Star 2M00014650+7009328, row 38
No good fits in 3 sigma of erros. Star 2M00015905+0052119, row 40
No good fits in 3 sigma of erros. Star 2M00045146+7010262, row 70
No good fits in 3 sigma of erros. Star 2M00053477+7129567, row 75
No good fits in 3 sigma of erros. Star 2M00073858+0010448, row 91
No good fits in 3 sigma of erros. Star 2M00080678+0057034, row 100
No good fits in 3 sigma of erros. Star 2M00083572+0043587, row 107
No good fits in 3 sigma of erros. Star 2M00083769+7149289, row 108
No good fits in 3 sigma of erros. Star 2M00092789+0145417, row 116
No good fits in 3 sigma of erros. Star 2M00131968+6619142, row 156
No good fits in 3 sigma of erros. Star 2M00132433+6716566, row 163
Done with 200.000000
No good fits in 3 sigma of erros. Star 2M00161756+7008036, row 228
No good fits in 3 sigma of err

#### 7. Add Radial velocity measurments
Add radial velocity measurments, errors, and date's observed to the all_average table. That way when we find the binary fraction we only need one table

In [12]:
all_average_rv = dsimp.rv_table_add(all_average, all_visit)

In [13]:
all_average_rv

RADIALV,RADIAL_ERR,RADIAL_DATE,APSTAR_ID,TARGET_ID,ASPCAP_ID,FILE,APOGEE_ID,TELESCOPE,LOCATION_ID,FIELD,J,J_ERR,H,H_ERR,K,K_ERR,RA,DEC,GLON,GLAT,APOGEE_TARGET1,APOGEE_TARGET2,APOGEE_TARGET3,TARGFLAGS,SURVEY,NINST [3],NVISITS,COMBTYPE,COMMISS,SNR,STARFLAG,STARFLAGS,ANDFLAG,ANDFLAGS,VHELIO_AVG,VSCATTER,VERR,VERR_MED,OBSVHELIO_AVG,OBSVSCATTER,OBSVERR,OBSVERR_MED,SYNTHVHELIO_AVG,SYNTHVSCATTER,SYNTHVERR,SYNTHVERR_MED,RV_TEFF,RV_LOGG,RV_FEH,RV_ALPHA,RV_CARB,RV_CCFWHM,RV_AUTOFWHM,SYNTHSCATTER,STABLERV_CHI2 [2],STABLERV_RCHI2 [2],CHI2_THRESHOLD [2],STABLERV_CHI2_PROB [2],APOGEE2_TARGET1,APOGEE2_TARGET2,APOGEE2_TARGET3,MEANFIB,SIGFIB,SNREV,APSTAR_VERSION,ASPCAP_VERSION,RESULTS_VERSION,EXTRATARG,MIN_H,MAX_H,MIN_JK,MAX_JK,PARAM [9],FPARAM [9],"PARAM_COV [9,9]","FPARAM_COV [9,9]",TEFF,TEFF_ERR,LOGG,LOGG_ERR,VMICRO,VMACRO,VSINI,M_H,M_H_ERR,ALPHA_M,ALPHA_M_ERR,ASPCAP_CHI2,ASPCAP_CLASS,ASPCAPFLAG,ASPCAPFLAGS,PARAMFLAG [9],FELEM [26],FELEM_ERR [26],X_H [26],X_H_ERR [26],X_M [26],X_M_ERR [26],C_FE,CI_FE,N_FE,O_FE,NA_FE,MG_FE,AL_FE,SI_FE,P_FE,S_FE,K_FE,CA_FE,TI_FE,TIII_FE,V_FE,CR_FE,MN_FE,FE_H,CO_FE,NI_FE,CU_FE,GE_FE,RB_FE,Y_FE,ND_FE,C_FE_ERR,CI_FE_ERR,N_FE_ERR,O_FE_ERR,NA_FE_ERR,MG_FE_ERR,AL_FE_ERR,SI_FE_ERR,P_FE_ERR,S_FE_ERR,K_FE_ERR,CA_FE_ERR,TI_FE_ERR,TIII_FE_ERR,V_FE_ERR,CR_FE_ERR,MN_FE_ERR,FE_H_ERR,CO_FE_ERR,NI_FE_ERR,CU_FE_ERR,GE_FE_ERR,RB_FE_ERR,Y_FE_ERR,ND_FE_ERR,C_FE_FLAG,CI_FE_FLAG,N_FE_FLAG,O_FE_FLAG,NA_FE_FLAG,MG_FE_FLAG,AL_FE_FLAG,SI_FE_FLAG,P_FE_FLAG,S_FE_FLAG,K_FE_FLAG,CA_FE_FLAG,TI_FE_FLAG,TIII_FE_FLAG,V_FE_FLAG,CR_FE_FLAG,MN_FE_FLAG,FE_H_FLAG,CO_FE_FLAG,NI_FE_FLAG,CU_FE_FLAG,GE_FE_FLAG,RB_FE_FLAG,Y_FE_FLAG,ND_FE_FLAG,ELEM_CHI2 [26],ELEMFLAG [26],REDUCTION_ID,SRC_H,WASH_M,WASH_M_ERR,WASH_T2,WASH_T2_ERR,DDO51,DDO51_ERR,IRAC_3_6,IRAC_3_6_ERR,IRAC_4_5,IRAC_4_5_ERR,IRAC_5_8,IRAC_5_8_ERR,IRAC_8_0,IRAC_8_0_ERR,WISE_4_5,WISE_4_5_ERR,TARG_4_5,TARG_4_5_ERR,AK_TARG,AK_TARG_METHOD,AK_WISE,SFD_EBV,WASH_DDO51_GIANT_FLAG,WASH_DDO51_STAR_FLAG,PMRA,PMDEC,PM_SRC,ALL_VISITS,VISITS,ALL_VISIT_PK [50],VISIT_PK [50],"FPARAM_CLASS [23,9]",CHI2_CLASS [23],ISO_MEANM,ISO_MEDIANM,ISO_STDM,ISO_MEANL,ISO_MEDIANL,ISO_STDL,ISO_MEANR
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,solMass,solMass,solMass,solLum,solLum,solLum,solRad
object,object,object,str45,str34,str46,str38,str18,str8,int16,str16,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,int32,int32,int32,str140,str14,int32,int32,int32,int16,float32,int32,str165,int32,str74,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,float32,float32,float32,str5,str4,str6,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str5,int32,str183,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float32,int32,str23,str12,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str17,float32,float32,int16,int16,float32,float32,str20,str845,str804,int32,int32,float32,float32,float64,float64,float64,float64,float64,float64,f

#### 8 Pickle the result
You cannot save the result all_average_rv like you would a normal fits table, because the new "RADIAL..." rows are column type 'object' and you can't save a fits file that has type object. But I've found that using pickle does the job just as well.

In [15]:
import pickle
import os
outfile = "all_average_rv_example.pck" # Name of the file to save
if os.path.exists(outfile): os.remove(outfile) # If that file name already exists, get rid of it
pickle.dump(all_average_rv, open(outfile, "wb")) # Put the table in that file. 

Now you have a simplified astropy table that you can use with the programs in bayesian likelihood. 

See the other notebook for then finding the binary fraction.